# Word Sens Disambiguation
Ce projet étudiant se réalise dans le cadre du projet PolysEmY,  qui a pour but de désambiguïser un ensemble connu d'acronymes dont les différents sens sont également connus.<br/>
En essayant d'améliorer l'apprentissage des embeddings de mots, ce travail s'inspire en partie des travaux de <b>Huang et al. <i>Improving Word Representations via Global Context and Multiple Word Prototypes</i>, 2012</b>

In [1]:
import re
import math
import nltk
import random
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
from xml.dom.minidom import parse
from gensim.models import Word2Vec
from nltk.cluster.kmeans import KMeansClusterer

trial_corpus_path = "trial_corpus.xml"
test_corpus_path = "test_corpus.xml"

Fournis une méthode pour charger un corpus au format xml

In [2]:
def loadCorpus(path):
    """Load a formatted corpus data file

    Parameters
    ----------
    path: str
        Path to the corpus xml file to load


    Returns
    -------
    list
        A 3 dimensional list containing for each document, the sentences that it is composed of.
        Where each sentence is a list of single tokens.
    dict
        A dictionnary mapping a lemma with it's document/sentence/index position and the BabelNet Sense attributed
    """

    DOMTree = parse(path)

    documents = []
    sens_dict = {}
    for doc in DOMTree.getElementsByTagName("document"):
        # For each document
        sentences = []
        for sent in doc.getElementsByTagName("sentence"):
            # And for each sentence
            # Append the new sentence
            s = sent.getAttribute("s")
            sentences.append(s.split())

            # Map the lemmas in the sentence with it's doc/sentence/index position and BabelNet sense
            for lem in sent.getElementsByTagName("lemma"):
                idx = lem.getAttribute("idx")
                lemma = lem.getAttribute("lemma")
                # Few lemma may have more than 1 BabelNet sense (due to redundancy in BN)
                # Only keep the 1st one
                sense = lem.getAttribute("senses").split()[0] 
                
                ctx = (int(doc.getAttribute("id")),
                        int(sent.getAttribute("id")),
                        int(idx),
                        sense)
                if not lemma in sens_dict:
                    sens_dict[lemma] = [ctx]
                else:
                    sens_dict[lemma].append(ctx)

        documents.append(sentences)

    return (documents, sens_dict)


documents, sens_dict = loadCorpus(trial_corpus_path)


print("Documents (%d):"%(len(documents)))
for i, d in enumerate(documents):
    print("\tDoc %2d: %02d sentences"%(i, len(d)))

print("\nLemmas (%d):"%(len(sens_dict)))
for i, (k, v) in zip(range(10), sens_dict.items()):
    print("\tLemma %d: %s -> %d values"%(i, k, len(v)))
print("\t...\n")
print("\nEx. lemma 0: ")
print("\t", list(sens_dict.keys())[0], "->", list(sens_dict.values())[0])

Documents (1):
	Doc  0: 36 sentences

Lemmas (124):
	Lemma 0: guerre_contre_la_drogue -> 1 values
	Lemma 1: Amérique_Latine -> 2 values
	Lemma 2: presse -> 9 values
	Lemma 3: mois -> 2 values
	Lemma 4: journaliste -> 5 values
	Lemma 5: trafiquant_de_drogue -> 2 values
	Lemma 6: guérillero -> 1 values
	Lemma 7: gauche -> 2 values
	Lemma 8: personne -> 1 values
	Lemma 9: Colombie -> 4 values
	...


Ex. lemma 0: 
	 guerre_contre_la_drogue -> [(0, 0, 7, 'bn:00028885n')]


# Méthode 1 : méthode «Huang»

## Implémentation de Word2vec pour l'apprentissage «naïf» d'embedding

In [3]:
def documentsSentences(doc):
    docSentences = []
    for d in doc:
        for s in d:
            docSentences.append(s)
    return docSentences

def createW2vModel():
    return Word2Vec(documentsSentences(documents), min_count=1)

w2v = createW2vModel()
w2v.save("w2v.model")

print("w2v Vocab size:", len(w2v.wv.vocab))

w2v Vocab size: 429


## Apprentissage d'embeddings par la méthode de Huang et al.

In [4]:
def lemma2Senses(lemma):
    senses = [bn for _,_,_,bn in sens_dict[lemma]]
    return list(set(senses))

print("\nEx.", list(sens_dict.keys())[0])
print("\t->", list(sens_dict.values())[0])
print("\t->", lemma2Senses(list(sens_dict.keys())[0]))


Ex. guerre_contre_la_drogue
	-> [(0, 0, 7, 'bn:00028885n')]
	-> ['bn:00028885n']


### Visualisation rapide des lemmes à désambiguïser

On remarque que peu de lemmes sont associés à plusieurs sens. Certains apparaissent plusieurs fois avec toujours le meme sens. Pire ! D'autres n'apparaissent qu'une seule fois.<br/>

Il est aussi intéressant de remarquer que certains lemmes sont associés 9 fois avec le sens_1 et 1 fois avec le sens_2. Ceci peut trouver son origine dans les annotations via BabelNet qui propose différents sens redondants d'un mot.<br/>
Pour exemple, le lemme <i>journaliste</i> est associé aux sens BabelNet suivants :
<ol>
    <li>bn:00048461n : celui qui recueille, écrit ou distribue des informations</li>
    <li>bn:00057562n : celui qui enquête, rapporte ou rédige les actualités</li>
</ol>

In [5]:
polysem = {}
solo = {}
npolysem = {}

for k,v in sens_dict.items():
    if len(v) == 1:
        solo[k] = v
        continue
    
    _,_,_,sense_bn = v[0]
    poly = False
    for _,_,_,bn in v:
        if bn != sense_bn:
            polysem[k] = v
            poly = True
            break
    if not poly:
        npolysem[k] = v

print("Nombre de lemmes à désambiguïser :", len(sens_dict))
print()

print("Lemmes ayant plusieurs sens :", len(polysem))
for _, (k, v) in zip(range(5), polysem.items()):
    print("\t", k, " -> ", v)

print("\nLemmes présents une seule fois :", len(solo))
for _, (k, v) in zip(range(5), solo.items()):
    print("\t", k, " -> ", v)

print("\nLemmes n'ayant qu'un seul sens :", len(npolysem))
for _, (k, v) in zip(range(5), npolysem.items()):
    print("\t", k, " -> ", v)

Nombre de lemmes à désambiguïser : 124

Lemmes ayant plusieurs sens : 3
	 journaliste  ->  [(0, 1, 8, 'bn:00048461n'), (0, 2, 10, 'bn:00048461n'), (0, 16, 43, 'bn:00048461n'), (0, 18, 38, 'bn:00057562n'), (0, 19, 27, 'bn:00048461n')]
	 contrôle  ->  [(0, 3, 29, 'bn:00022287n'), (0, 26, 19, 'bn:00022283n')]
	 journal  ->  [(0, 5, 30, 'bn:00057563n'), (0, 6, 5, 'bn:00057563n'), (0, 7, 11, 'bn:00057563n'), (0, 8, 19, 'bn:00057564n'), (0, 18, 5, 'bn:00057563n')]

Lemmes présents une seule fois : 86
	 guerre_contre_la_drogue  ->  [(0, 0, 7, 'bn:00028885n')]
	 guérillero  ->  [(0, 1, 22, 'bn:02557244n')]
	 personne  ->  [(0, 1, 33, 'bn:00046516n')]
	 année  ->  [(0, 2, 5, 'bn:00078738n')]
	 août  ->  [(0, 3, 18, 'bn:00007140n')]

Lemmes n'ayant qu'un seul sens : 35
	 Amérique_Latine  ->  [(0, 0, 9, 'bn:00050165n'), (0, 4, 14, 'bn:00050165n')]
	 presse  ->  [(0, 0, 23, 'bn:00064245n'), (0, 4, 38, 'bn:00064245n'), (0, 12, 4, 'bn:00064245n'), (0, 16, 29, 'bn:00064245n'), (0, 17, 19, 'bn:0006424

On remarque qu'au final peu de lemmes dans les corpus de dev et test possèdent plusieurs sens dans les corpus. Certains ne sont présents qu'une fois pendant que d'autre n'apparaisse tout le temps qu'avec le meme sens.<br/>
- Il est a noter que ce n'est pas parce qu'un mot apparait toujours avec le meme sens dans le corpus qu'il n'est polysémique.

## Implémentation de la méthode proposée par Huang
<ol>
    <li>Collecter les contextes d'occurrence d'un mot a désambiguïser</li>
    <li>Calculer un vecteur de contexte, moyenne des embedding de chaque mots dans le contexte</li>
    <li>Clusterer les vecteurs de contextes, spherical K-means</li>
    <li>Associe à chaque cluster un sens, le centroïde du cluster est l'embedding du sens du mot</li>
</ol>

In [6]:
def meanSentenceVector(w2vModel, sentence):
    return np.array([w2vModel.wv[word] for word in sentence]).mean(axis=0)

def sumSentenceVector(w2vModel, sentence):
    return np.array([w2vModel.wv[word] for word in sentence]).sum(axis=0)

In [7]:
def extractWindow(sentence, start, end):
    return sentence[max(0, start) : min(len(sentence), end)+1]

print(extractWindow("Je suis très content de manger une pomme !".split(), 1, 5))
print(extractWindow("Je suis très content de manger une pomme !".split(), -1, 5))
print(extractWindow("Je suis très content de manger une pomme !".split(), 1, 15))

['suis', 'très', 'content', 'de', 'manger']
['Je', 'suis', 'très', 'content', 'de', 'manger']
['suis', 'très', 'content', 'de', 'manger', 'une', 'pomme', '!']


In [8]:
ctx_w = 11 # Contexte window size

global_truth = []
global_classif = []

for lemma, senses in polysem.items():
    labels = lemma2Senses(lemma)
    num_senses = len(labels)

    # Map clusters with a value from, 0 to num_senses-1
    truth = [labels.index(bn) for _,_,_,bn in senses]
    global_truth.append(truth)

    mean_vectors = []
    for d,s,i,_ in senses:
        l = len(documents[d][s])
        # Extract the words in the contexte window
        window = extractWindow(documents[d][s], i-math.floor((ctx_w-1)/2), i+math.ceil((ctx_w-1)/2))

        # Compute the context vector (mean of the words vectors in the window)
        mean_vectors.append(meanSentenceVector(w2v, window))

    # Spherical K-means clustering
    skm = KMeansClusterer(num_senses, nltk.cluster.util.cosine_distance, rng=random.Random(0), repeats=10)
    assigned_clusters = skm.cluster(mean_vectors, assign_clusters=True)
    global_classif.append(assigned_clusters)

    #print(truth)
    #print(assigned_clusters)
    #print()

### Associe à chaque cluster un sens

In [9]:
def argsmax(lst):
    max = lst[0]
    argsmax = []
    for i in range(len(lst)):
        if lst[i] == max:
            argsmax.append(i)
        elif lst[i] > max:
            max = lst[i]
            argsmax = [i]
    return argsmax

def couple(arr1, arr2):
    # truth, classif
    l = len(arr1)
    nb_c = len(set(arr1))
    arr = np.zeros((nb_c,nb_c), dtype=int)

    for i in range(l):
        arr[arr2[i],arr1[i]] += 1

    cs = [i for i in range(len(arr))]
    ts = [i for i in range(len(arr))]
    map = {}

    for iter in range(len(arr)):
        temp = len(arr)
        for i in range(len(arr)):
            c = arr[i]
            ams = argsmax(c)
            if len(ams) == 1:
                ams = ams[0]
                map[cs[i]] = ts[ams]
                arr = np.delete(np.delete(arr, ams, 1), i, 0)
                cs = np.delete(cs, i)
                ts = np.delete(ts, ams)
                break
        
        if len(arr) == temp:
            j = np.argmax(arr[0])
            map[cs[0]] = ts[j]
            arr = np.delete(np.delete(arr, j, 1), 0, 0)
            cs = np.delete(cs, 0)
            ts = np.delete(ts, j)

    return map

couple(global_truth[0], global_classif[0])

{0: 0, 1: 1}

## Premiers Résultats (dev)
Pour chaque lemme affiche :
- La position d'apparission dans le corpus de chaque occurence ainsi que l'id BabelNet de sa définition. Pour une occurence, on a un tuple (n° document, n° phrase, position dans la phrase, id BabelNet).
- La prédiction (résulat du clustering). Chaque occurence est associée à un cluster.
- La target, ou chaque occurence est associée à son sens réel.

À noter que le nombre de sens d'un mot se base qur les données de SemEval, le corpus définit pour chaque mot à désambiguiser entre 1 et 3 sens. Dans ces résultats lorsque l'on voit "Prédiction : 0, 1", cela signifit que la premiere occurence est associée au sens 0 du mot, la deuxieme au sens 1 du mot. Les sens 0 ou 1 sont ceux explicité précedemment, définit par le corpus SemEval.

In [10]:
final_classif = []
for i in range(len(global_classif)):
    cluster_tags = couple(global_truth[i], global_classif[i])
    final_classif.append([cluster_tags[i] for i in global_classif[i]])

    print("Lemme\t\t", list(polysem.keys())[i])
    print("Pos & id\t", list(polysem.values())[i])
    print("Prédiction\t", final_classif[i])
    print("Réalité\t\t", global_truth[i])
    print()

Lemme		 journaliste
Pos & id	 [(0, 1, 8, 'bn:00048461n'), (0, 2, 10, 'bn:00048461n'), (0, 16, 43, 'bn:00048461n'), (0, 18, 38, 'bn:00057562n'), (0, 19, 27, 'bn:00048461n')]
Prédiction	 [0, 1, 0, 1, 0]
Réalité		 [0, 0, 0, 1, 0]

Lemme		 contrôle
Pos & id	 [(0, 3, 29, 'bn:00022287n'), (0, 26, 19, 'bn:00022283n')]
Prédiction	 [1, 0]
Réalité		 [1, 0]

Lemme		 journal
Pos & id	 [(0, 5, 30, 'bn:00057563n'), (0, 6, 5, 'bn:00057563n'), (0, 7, 11, 'bn:00057563n'), (0, 8, 19, 'bn:00057564n'), (0, 18, 5, 'bn:00057563n')]
Prédiction	 [1, 1, 1, 1, 0]
Réalité		 [0, 0, 0, 1, 0]



In [11]:
def precision(pred, real):
    somme = 0
    precision = 0
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            somme += 1
            if pred[i][j] == real[i][j]:
                precision += 1
    return precision/somme

res = precision(final_classif, global_truth)

print("Taux de succès de prédiction : {:.2}%".format(res))
print("Taux d'erreurs de prédiction : {:.2}%".format(1-res))

Taux de succès de prédiction : 0.67%
Taux d'erreurs de prédiction : 0.33%


In [12]:
#print(sklearn.metrics.classification_report([y for x in global_truth for y in x], [y for x in final_classif for y in x]))

## Résultats sur le test
Réamplique la procédure, cette fois-ci sur le test.

In [13]:
documents, sens_dict = loadCorpus(test_corpus_path)

w2v = createW2vModel()

In [14]:
global_truth = []
global_classif = []

for lemma, senses in sens_dict.items():
    labels = lemma2Senses(lemma)
    num_senses = len(labels)

    # Map clusters with a value from, 0 to num_senses-1
    truth = [labels.index(bn) for _,_,_,bn in senses]
    global_truth.append(truth)

    mean_vectors = []
    for d,s,i,_ in senses:
        l = len(documents[d][s])
        # Extract the words in the contexte window
        window = extractWindow(documents[d][s], i-math.floor((ctx_w-1)/2), i+math.ceil((ctx_w-1)/2))

        # Compute the context vector (mean of the words vectors in the window)
        mean_vectors.append(meanSentenceVector(w2v, window))

    # Spherical K-means clustering
    skm = KMeansClusterer(num_senses, nltk.cluster.util.cosine_distance, rng=random.Random(0), repeats=10)
    assigned_clusters = skm.cluster(mean_vectors, assign_clusters=True)
    global_classif.append(assigned_clusters)

final_classif = []
for i in range(len(global_classif)):
    cluster_tags = couple(global_truth[i], global_classif[i])
    final_classif.append([cluster_tags[i] for i in global_classif[i]])

In [15]:
res = precision(final_classif, global_truth)

print("Taux de succès de prédiction : {:.3}%".format(res))
print("Taux d'erreurs de prédiction : {:.3}%".format(1-res))

Taux de succès de prédiction : 0.935%
Taux d'erreurs de prédiction : 0.0653%


## Remarque
On peut observer un score très honorable de 93% de bonne prédiction. Cependant ce score est biaisé par les nombreux mot n'ayant qu'un seul sens.

In [16]:
polysem = {}
solo = {}
npolysem = {}

for k,v in sens_dict.items():
    if len(v) == 1:
        solo[k] = v
        continue
    
    _,_,_,sense_bn = v[0]
    poly = False
    for _,_,_,bn in v:
        if bn != sense_bn:
            polysem[k] = v
            poly = True
            break
    if not poly:
        npolysem[k] = v

somme = len(polysem) + len(solo) + len(npolysem)

print("Lemmes ayant plusieurs sens : {} ({:2.2}%)".format(len(polysem), len(polysem)/somme*100))
print("\nLemmes présents une seule fois : {} ({:2.3}%)".format(len(solo), len(solo)/somme*100))
print("\nLemmes n'ayant qu'un seul sens : {} ({:2.3}%)".format(len(npolysem), len(npolysem)/somme*100))

Lemmes ayant plusieurs sens : 69 (9.3%)

Lemmes présents une seule fois : 432 (58.0%)

Lemmes n'ayant qu'un seul sens : 244 (32.8%)


## Remarque
On voit ici que le problème de desambiguïsation n'a d'interet que sur seulement 9% du corpus.*

## Résultats sur le test bis (sans tenir compte des mots non polysémiques)

In [17]:
global_truth = []
global_classif = []

for lemma, senses in polysem.items():
    labels = lemma2Senses(lemma)
    num_senses = len(labels)

    # Map clusters with a value from, 0 to num_senses-1
    truth = [labels.index(bn) for _,_,_,bn in senses]
    global_truth.append(truth)

    mean_vectors = []
    for d,s,i,_ in senses:
        l = len(documents[d][s])
        # Extract the words in the contexte window
        window = extractWindow(documents[d][s], i-math.floor((ctx_w-1)/2), i+math.ceil((ctx_w-1)/2))

        # Compute the context vector (mean of the words vectors in the window)
        mean_vectors.append(meanSentenceVector(w2v, window))

    # Spherical K-means clustering
    skm = KMeansClusterer(num_senses, nltk.cluster.util.cosine_distance, rng=random.Random(0), repeats=10)
    assigned_clusters = skm.cluster(mean_vectors, assign_clusters=True)
    global_classif.append(assigned_clusters)

final_classif = []
for i in range(len(global_classif)):
    cluster_tags = couple(global_truth[i], global_classif[i])
    final_classif.append([cluster_tags[i] for i in global_classif[i]])

res = precision(final_classif, global_truth)

print("Taux de succès de prédiction : {:.3}%".format(res))
print("Taux d'erreurs de prédiction : {:.3}%".format(1-res))

Taux de succès de prédiction : 0.68%
Taux d'erreurs de prédiction : 0.32%


Les résultats sont cette fois-ci plus réalistes avec 68% de bonne prédictions

# Méthode 2, méthode à base de définitions
calule des similarités entre une occurence d'un mot et ses définitions

In [18]:
#import xml.etree.ElementTree as et

output_classif_path = "test_corpus.classif"

documents, sens_dict = loadCorpus(trial_corpus_path)

Charge le dictionnaire

In [19]:
def load_dictionary(path):
    dictionary_dict = {}

    with open(path, "r") as file:
        file.readline()
        for row in file:
            row = row.split(";")
            lemma = row.pop(0)
            nb = int(row.pop(0))
            
            # Make sure the definition is not empty...
            if nb > 0:
                ids = (row.pop(0)).split(",")
                defs = (";".join(row)).split("\",\"")
                temp = defs[0].split(",\"")
                defs = temp + defs[1:]

                dictionary_dict[lemma] = (ids, defs)
    
    return dictionary_dict

dictionary = load_dictionary("dict.dictionary")
print("Dictionary length:", len(dictionary))

Dictionary length: 949


### Exemple du contenu du dictionnaire pour le mot "mardi"

In [20]:
dictionary["mardi"]

(['bn:00078546n', 'bn:00118342n'],
 ['"Le mardi est le jour de la semaine qui succède au lundi et qui précède le mercredi. Jour de la semaine Le deuxième jour de la semaine en Europe et dans les pays utilisant la norme ISO 8601; le troisième jour de la semaine aux États-Unis d\'Amérique.',
  'Mardi est le troisième livre publié par l\'écrivain américain Herman Melville."\n'])

## Word2Vec
Apprend les embeddings sur le corpus et les definitions du dictionnaire

In [21]:
def definitionsSentences():
    defSentences = []
    for _,defs in dictionary.values():
        for s in defs:
            defSentences.append(s.split())
    return defSentences

def createExtendedW2vModel():
    return Word2Vec(documentsSentences(documents)+definitionsSentences(), min_count=1)

w2v = createExtendedW2vModel()

print("Extended w2v Vocab size:", len(w2v.wv.vocab))

Extended w2v Vocab size: 25722


## Calcule des vecteurs de définitions
Un vecteur de définition est la moyenne des embeddings des mots d'une définition

In [22]:
defsVectors = {} #map id lemma -> vecteur

for k,(ids,defs) in dictionary.items():
    defsVectors[k] = np.array([meanSentenceVector(w2v,d.split()) for d in defs]).reshape((len(defs), 100))

## Test association mot->sens
Associe un vecteur de contexte avec le vecteur de définition le plus proche.<br/>
La notion de "proche" se traduit par une similarité cosine.<br/>

### Corpus de dev
Exemple d'association mot->sens pour les occurences du mots journal sur le dev

In [23]:
mean_vectors = []
for d,s,p,bn in sens_dict["journal"]:
    window = extractWindow(documents[d][s], p-math.floor((ctx_w-1)/2), p+math.ceil((ctx_w-1)/2))
    mean_vectors.append(meanSentenceVector(w2v, window))

# Occurences du mot journal dans le corpus de dev
journaux = [(0, 5, 30, 'bn:00057563n'), (0, 6, 5, 'bn:00057563n'), (0, 7, 11, 'bn:00057563n'), (0, 8, 19, 'bn:00057564n'), (0, 18, 5, 'bn:00057563n')]

for i in range(len(mean_vectors)):
    csim = w2v.wv.cosine_similarities(mean_vectors[i], defsVectors["journal"])
    amax = np.argmax(csim)
    d,s,p,bn = journaux[i]
    print("---- Phrase :\n\t", " ".join(documents[d][s]))
    print("\tDocument %d, phrase %d à la position %d (BabelNet : %s)"%(d, s, p, bn))
    print("---- Sortie :")
    print("\tid BabelNet prédit : ", dictionary["journal"][0][amax])
    print("\tDéfinition prédite : ", dictionary["journal"][1][amax])
    print("Similarité cosine : ", csim[amax])
    print("\n\n")

---- Phrase :
	 Le mardi , les participants à la conférence ont été informés d' une autre atrocité , l' assassinat à Medellin de deux employés d' El_Espectador , le deuxième plus grand journal de Colombie .
	Document 0, phrase 5 à la position 30 (BabelNet : bn:00057563n)
---- Sortie :
	id BabelNet prédit :  bn:00057563n
	Définition prédite :  L'expression presse écrite désigne, d'une manière générale, l'ensemble des moyens de diffusion de l’information écrite, ce qui englobe notamment les journaux quotidiens, les publications périodiques et les organismes professionnels liés à la diffusion de l'information. Périodique présentant des nouvelles et autres informations diverses Publication (en général publiée chaque semaine ou chaque mois et imprimé sur du papier de basse qualité) qui contient des actualités et d'autres articles. Journal qui est publié dans la soirée. Publication publiée une ou deux fois par jour (en général imprimé sur un papier de faible qualité) qui contient les actuali

### Corpus de test
Le résultat est disponible dans le fichier "test_corpus.classif"<br/>
Le dictionnaire n'étant pas parfait, la procédure suivante peut échouer à trouver une définition à un mot. Cale est soit dû au fait que le dictionnaire n'associe aucune définition au mot ou bien dû à la façon dont la recherche s'effectue dans le dictionnaire (sensibilité de la casse, format différent, tirets...)

In [24]:
documents, sens_dict = loadCorpus(test_corpus_path)

w2v = createExtendedW2vModel()
w2v.save("w2v_extended.model")

with open(output_classif_path, "w") as file:
    file.write("lemma;doc_id;sent_id;sent_pos;output_bn_id;output_def\n")

    for lemma,v in sens_dict.items():
        for d,s,p,bn in v:
            window = extractWindow(documents[d][s], p-math.floor((ctx_w-1)/2), p+math.ceil((ctx_w-1)/2))
            mean_vector = meanSentenceVector(w2v, window)
            
            lemma = lemma.lower()
            if lemma in defsVectors:
                csim = w2v.wv.cosine_similarities(mean_vector, defsVectors[lemma])

                amax = np.argmax(csim)

                bn_id = dictionary[lemma][0][amax]
                bn_def = dictionary[lemma][1][amax]

                file.write("{};{};{};{};{};\"{}\"\n".format(lemma, d, s, p, bn_id, bn_def))
            else:
                print("Missing definition for \"%s\""%(lemma))

Missing definition for "récrimination"
Missing definition for "washington"
Missing definition for "yvo_de_boer"
Missing definition for "union_européenne"
Missing definition for "altercation"
Missing definition for "administration_obama"
Missing definition for "délégué"
Missing definition for "délégué"
Missing definition for "palais_des_congrès"
Missing definition for "politique_mondiale"
Missing definition for "caja_laboral"
Missing definition for "nasdaq"
Missing definition for "nasdaq"
Missing definition for "europe"
Missing definition for "europe"
Missing definition for "irak"
Missing definition for "irak"
Missing definition for "irak"
Missing definition for "irak"
Missing definition for "irak"
Missing definition for "irak"
Missing definition for "internet"
Missing definition for "internet"
Missing definition for "instabilité"
Missing definition for "twitter"
Missing definition for "twitter"
Missing definition for "twitter"
Missing definition for "twitter"
Missing definition for "tw

## Discussion
Les résultats obtenus sont tres interessant mais extrement dépendant de la qualité des définitions BabelNet ainsi que de leur nombre.<br/>
- Il parrait assez evidant que plus un mot est polysemique et plus la méthode a de chance de se tromper de sens (certains mots comme "état" et "liberté" ont 21 sens BabelNet)
- La redondance des définitons BabelNet ne permet pas de simplement évaluer les définitions une à une et en tirer un score de classification. Par exemple, pour le mot "journaliste", sur les 6 concepts, 4 définissent le meme concept de "journaliste de presse".
- La représentation vectorielle d'une définission est dépendante de la qualité des définitions BabelNet qui sont parfois très succinctes voire inexistantes.